# fix_geojson.ipynb

merged_polygon_with_farm_pin.geojsonのうち、耕作者情報(FarmerIndicationNumberHash)が存在するものだけ抽出して新しくgeojsonファイルを生成する。(軽量化のため)

In [ ]:
import geopandas as gpd

# Load the GeoJSON file
file_path = '../src/models/area_optimization/data/processed/merged_geojson/merged_polygon_with_farm_pin.geojson'
gdf = gpd.read_file(file_path)

# Filter out rows where FarmerIndicationNumberHash is null
filtered_gdf = gdf[gdf['FarmerIndicationNumberHash'].notnull()]

filtered_gdf.head()

,polygon_uuid,land_type,issue_year,edit_year,history,last_polygon_uuid,prev_last_polygon_uuid,local_government_cd,point_lng,point_lat,...,Address,Tiban,SectionOfPolygon,SectionOfPolygonCodeName,DaichoId,daicho_shubetsu_cd,CommencementDate,EndStagesDate,UseIntentionInvestigationDate,geometry
651,91b26c2f-edf8-4e24-8cb6-c49fc801f6c4,100,2024,2020,"[ { ""筆ポリゴンID"": ""24ace6aa-ddbd-4a3b-8771-06440a...",24ace6aa-ddbd-4a3b-8771-06440ae9a976,92790549-ed97-4924-bfb4-6343eca829a7,082015,140.375388,36.377329,...,茨城県水戸市大足町1755-1,1755-1,3,その他,81006825,KNO01,None,None,None,"POLYGON ((140.37559 36.37763, 140.37579 36.377..."
652,f377b08b-cdfa-409a-ad98-aff6322d97b4,100,2024,2020,"[ { ""筆ポリゴンID"": ""35f7e3c9-6f9c-481b-ada1-e95502...",35f7e3c9-6f9c-481b-ada1-e955026d6562,8c530a77-b5a8-4f22-9ea6-608d310d11ca,082015,140.375710,36.377958,...,茨城県水戸市大足町1777,1777,3,その他,81006854,KNO01,20190301,20290331,None,"POLYGON ((140.37527 36.37798, 140.37576 36.378..."
653,0d4ae71a-76f0-429a-a735-1678d8718c78,100,2024,2020,"[ { ""筆ポリゴンID"": ""41b32703-c783-4f42-8664-01233b...",41b32703-c783-4f42-8664-01233b4aa040,a87e2afb-6780-4a61-829c-3acfae4c7128,082015,140.375397,36.378252,...,茨城県水戸市大足町1776,1776,3,その他,81006853,KNO01,20180101,20281231,None,"POLYGON ((140.37553 36.37851, 140.37574 36.378..."
654,12be4182-b2c2-4255-a693-369ea88376b4,100,2024,2020,"[ { ""筆ポリゴンID"": ""a72ac7a9-a854-416a-964c-69e637...",a72ac7a9-a854-416a-964c-69e637561231,057d37b7-45d3-4e92-80b9-556302361677,082015,140.376069,36.377547,...,茨城県水戸市大足町字中ノ割1778,1778,3,その他,81007080,KNO01,None,None,None,"POLYGON ((140.37602 36.37722, 140.37561 36.377..."
655,3f07bdee-c4ed-4e4e-bbd6-c8d92e401743,100,2024,2020,"[ { ""筆ポリゴンID"": ""574fafa4-6e0a-43fc-95a4-45c63d...",574fafa4-6e0a-43fc-95a4-45c63d522e67,060c7d6b-eac7-47a9-bea9-86b8fde9bfce,082015,140.375288,36.381043,...,茨城県水戸市大足町1837,1837,3,その他,81006911,KNO01,None,None,None,"POLYGON ((140.37533 36.38139, 140.37571 36.381..."


In [ ]:
# print(filtered_gdf.iloc[0].geometry)

POLYGON ((140.375593576 36.377632174, 140.37579015100002 36.377416549, 140.37515833400002 36.377016351, 140.37500107 36.377251159, 140.375593576 36.377632174))


In [20]:
# Save the filtered GeoDataFrame to a new GeoJSON file
output_file_path = '../src/models/area_optimization/data/processed/merged_geojson/filtered_merged_polygon_with_farm_pin.geojson'
filtered_gdf.to_file(output_file_path, driver='GeoJSON')
print('Filtered GeoJSON file saved to:', output_file_path)


Filtered GeoJSON file saved to: ../src/models/area_optimization/data/processed/merged_geojson/filtered_merged_polygon_with_farm_pin.geojson


In [18]:
print(filtered_gdf.iloc[0]['geometry'])

POLYGON ((140.375593576 36.377632174, 140.37579015100002 36.377416549, 140.37515833400002 36.377016351, 140.37500107 36.377251159, 140.375593576 36.377632174))


In [19]:
import geopandas as gpd
from shapely.geometry import Polygon

# サンプルのGeoDataFrameを作成
data = {'geometry': [Polygon([(-73.981, 40.768), (-73.981, 40.769), (-73.982, 40.769), (-73.982, 40.768)])]}
gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")  # EPSG:4326は緯度経度の座標系

# 投影法を変える前の面積を計算（球面積分を使用）
from pyproj import Geod
geod = Geod(ellps="WGS84")
def calculate_area(geom):
    lon, lat = geom.exterior.coords.xy
    return abs(geod.polygon_area_perimeter(lon, lat)[0])
gdf['area_spherical'] = gdf['geometry'].apply(calculate_area)

# 適切な投影法に変換（例: UTMゾーン18N）
gdf_projected = gdf.to_crs(epsg=32618)

# 投影法を変えた後の面積を計算
gdf_projected['area_projected'] = gdf_projected['geometry'].area

# 面積の比較
print("球面積分による面積:", gdf['area_spherical'].iloc[0])
print("投影法による面積:", gdf_projected['area_projected'].iloc[0])

球面積分による面積: 9375.77861827612
投影法による面積: 9369.98417634582
